In [16]:
import os
import pandas as pd
import json
import re
import time
from google import genai
from dotenv import load_dotenv

load_dotenv(override=True)

# Setup multiple API keys untuk fallback
API_KEYS = [
    os.getenv("GEMINI_API_KEY"),      # API Key 1 (primary)
    os.getenv("GEMINI_API_KEY_2")     # API Key 2 (fallback)
]

# Filter API keys yang valid (tidak None)
API_KEYS = [key for key in API_KEYS if key]

if not API_KEYS:
    raise ValueError("❌ Tidak ada API key yang valid! Set GEMINI_API_KEY atau GEMINI_API_KEY_2 di .env")

print(f"✅ Loaded {len(API_KEYS)} API key(s)")

# Initialize dengan API key pertama
current_api_key_index = 0
client = genai.Client(api_key=API_KEYS[current_api_key_index])

def switch_api_key():
    """Switch ke API key berikutnya"""
    global current_api_key_index, client
    
    if len(API_KEYS) <= 1:
        print("⚠️ Hanya ada 1 API key, tidak bisa switch")
        return False
    
    current_api_key_index = (current_api_key_index + 1) % len(API_KEYS)
    client = genai.Client(api_key=API_KEYS[current_api_key_index])
    print(f"🔄 Switch ke API key #{current_api_key_index + 1}")
    return True

✅ Loaded 2 API key(s)


In [17]:
df = pd.read_csv('data_filtered.csv')
print(f"Total komentar: {len(df)}")
print("\nContoh data awal:")
df.head()

Total komentar: 1072

Contoh data awal:


,Video_ID,Teks_Komentar,text_normalized_temp
0,MIo4tGN11j0,"Sempat mikir mau pindah ke negara sebelah, nge...","sempat mikir mau pindah ke negara sebelah, nge..."
1,MIo4tGN11j0,"Kalo kabur mau kemana ke Singapur ,emang di Si...","kalau kabur mau kemana ke singapur ,emang di s..."
2,MIo4tGN11j0,Klo sudah gelap susah terangnya lebih baik bubar,kalau sudah gelap susah terangnya lebih baik b...
3,MIo4tGN11j0,Siap siap ente motivator Indonesia gelap...dap...,siap siap ente motivator indonesia gelap...dap...
4,MIo4tGN11j0,"Diskusi yg segar,menarik dan bermutu... Antara...","diskusi yang segar,menarik dan bermutu... anta..."


In [18]:
for m in client.models.list():
    print(f"- {m.name}")

- models/embedding-gecko-001
- models/gemini-2.5-flash
- models/gemini-2.5-pro
- models/gemini-2.0-flash-exp
- models/gemini-2.0-flash
- models/gemini-2.0-flash-001
- models/gemini-2.0-flash-exp-image-generation
- models/gemini-2.0-flash-lite-001
- models/gemini-2.0-flash-lite
- models/gemini-2.0-flash-lite-preview-02-05
- models/gemini-2.0-flash-lite-preview
- models/gemini-exp-1206
- models/gemini-2.5-flash-preview-tts
- models/gemini-2.5-pro-preview-tts
- models/gemma-3-1b-it
- models/gemma-3-4b-it
- models/gemma-3-12b-it
- models/gemma-3-27b-it
- models/gemma-3n-e4b-it
- models/gemma-3n-e2b-it
- models/gemini-flash-latest
- models/gemini-flash-lite-latest
- models/gemini-pro-latest
- models/gemini-2.5-flash-lite
- models/gemini-2.5-flash-image-preview
- models/gemini-2.5-flash-image
- models/gemini-2.5-flash-preview-09-2025
- models/gemini-2.5-flash-lite-preview-09-2025
- models/gemini-3-pro-preview
- models/gemini-3-pro-image-preview
- models/nano-banana-pro-preview
- models/gemin

In [19]:
def classify_sentiment_batch(comments):
    global current_api_key_index, client  # Akses variabel global
    
    # Format JSON agar model lebih patuh struktur
    comments_input = [{"id": i+1, "text": c} for i, c in enumerate(comments)]
    comments_json = json.dumps(comments_input, ensure_ascii=False)

    prompt = f"""
    Bertindaklah sebagai Ahli Linguistik Sosial Indonesia yang spesialis mendeteksi "Sikap/Stance".
    
    Tugas: Tentukan sikap penulis komentar terhadap fenomena "KABUR DARI INDONESIA" (pindah kewarganegaraan/migrasi).
    
    KELAS LABEL:
    1. SUPPORT (Positif terhadap ide kabur): Setuju pindah, mengeluh soal negara (push factor), atau memuji luar negeri.
    2. REJECT (Negatif terhadap ide kabur): Tidak setuju pindah, membela negara, atau menyindir orang yang mau pindah.
    3. NEUTRAL: Bertanya, bingung, atau tidak ada opini jelas.

    CONTOH PEMBELAJARAN (FEW-SHOT):
    Input: "Gaji di sini cuma numpang lewat, mending jadi kuli di Jepang."
    Label: SUPPORT (Alasan: Mengeluh kondisi lokal, memuji opsi luar negeri)

    Input: "Halah, di luar negeri pajaknya juga gila kali, jangan mimpi."
    Label: REJECT (Alasan: Skeptis terhadap ide pindah)

    Input: "Emang syarat visa permanent resident Australia apa aja ya?"
    Label: NEUTRAL (Alasan: Hanya bertanya informasi)

    Input: "Makin cinta sama pemerintah, pajaknya mantap sekali ❤️"
    Label: SUPPORT (Alasan: Sarkasme, sebenarnya marah pada pemerintah -> ingin kabur)

    Input: "Katanya mau pindah tapi masih komen pakai bahasa Indo wkwk."
    Label: REJECT (Alasan: Menyindir orang yang mau pindah)

    TUGAS ANDA:
    Klasifikasikan daftar komentar berikut. 
    Berikan output HANYA dalam format JSON Murni seperti: [{{"id": 1, "label": "SUPPORT"}}, {{"id": 2, "label": "REJECT"}}, ...]
    Jangan tambahkan teks lain di luar JSON.
    
    DATA INPUT:
    {comments_json}
    """

    # Fallback strategy: coba beberapa model jika yang pertama gagal
    models_to_try = [
        "gemini-2.5-flash",           # Terbaru (mungkin quota terpisah)
        "gemini-2.0-flash-lite-001",  # Lite version (mungkin lebih generous)
        "gemini-2.0-flash-exp",       # Experimental
        "gemini-flash-latest"         # Latest alias
    ]
    
    last_error = None
    
    for model_name in models_to_try:
        # Coba setiap model dengan semua API key yang tersedia
        for attempt in range(len(API_KEYS)):
            try:
                response = client.models.generate_content(
                    model=model_name,
                    contents=prompt,
                    # Memaksa output JSON (Fitur Gemini 1.5/2.0 agar format tidak hancur)
                    config={
                        "response_mime_type": "application/json",
                        "temperature": 0.3  # Lebih deterministik
                    }
                )

                raw_output = response.text.strip()
                
                # Parsing JSON langsung (Jauh lebih aman daripada Regex)
                parsed_data = json.loads(raw_output)
                
                # Validasi urutan dan ambil labelnya saja
                final_labels = []
                # Label map dengan fallback case-insensitive
                label_map = {
                    "SUPPORT": "Positif", 
                    "REJECT": "Negatif", 
                    "NEUTRAL": "Netral",
                    "support": "Positif",
                    "reject": "Negatif",
                    "neutral": "Netral"
                }
                
                for i in range(len(comments)):
                    # Cari item dengan id yang sesuai (jaga-jaga kalau urutan output model ngaco)
                    item = next((x for x in parsed_data if x.get('id') == i+1), None)
                    if item and 'label' in item:
                        final_labels.append(label_map.get(item['label'], "Netral"))
                    else:
                        final_labels.append("Error")
                
                # Validasi jumlah
                if len(final_labels) != len(comments):
                    print(f"Jumlah label ({len(final_labels)}) ≠ jumlah komentar ({len(comments)})")
                    if len(final_labels) < len(comments):
                        final_labels += ["Error"] * (len(comments) - len(final_labels))
                    else:
                        final_labels = final_labels[:len(comments)]
                
                print(f"✅ Berhasil: {model_name} (API key #{current_api_key_index + 1})")
                return final_labels
            
            except json.JSONDecodeError as e:
                print(f"❌ {model_name}: Error parsing JSON - {e}")
                last_error = e
                break  # Parsing error bukan masalah API key, skip model ini
                
            except Exception as e:
                error_str = str(e)
                # Jika quota exhausted, coba switch API key
                if "RESOURCE_EXHAUSTED" in error_str or "429" in error_str:
                    print(f"⚠️ {model_name} (API key #{current_api_key_index + 1}): Quota exhausted")
                    last_error = e
                    
                    # Coba switch ke API key berikutnya
                    if attempt < len(API_KEYS) - 1:
                        if switch_api_key():
                            print(f"🔄 Retry {model_name} dengan API key baru...")
                            time.sleep(2)  # Brief pause before retry
                            continue
                    else:
                        print(f"❌ Semua API key habis untuk {model_name}, coba model lain...")
                        break
                else:
                    print(f"❌ {model_name}: Error - {e}")
                    last_error = e
                    break
        
        # Reset ke API key pertama untuk model berikutnya
        if current_api_key_index != 0:
            current_api_key_index = 0
            client = genai.Client(api_key=API_KEYS[0])
    
    # Jika semua model dan API key gagal
    print(f"❌ Semua kombinasi model & API key gagal. Error terakhir: {last_error}")
    return ["Error"] * len(comments)

In [20]:
comments = df['Teks_Komentar'].head().tolist()
labels = classify_sentiment_batch(comments)

df_preview = pd.DataFrame({
    "Teks_Komentar": comments,
    "sentiment": labels
})

df_preview

⚠️ gemini-2.5-flash (API key #1): Quota exhausted
🔄 Switch ke API key #2
🔄 Retry gemini-2.5-flash dengan API key baru...
✅ Berhasil: gemini-2.5-flash (API key #2)


,Teks_Komentar,sentiment
0,"Sempat mikir mau pindah ke negara sebelah, nge...",Negatif
1,"Kalo kabur mau kemana ke Singapur ,emang di Si...",Negatif
2,Klo sudah gelap susah terangnya lebih baik bubar,Positif
3,Siap siap ente motivator Indonesia gelap...dap...,Negatif
4,"Diskusi yg segar,menarik dan bermutu... Antara...",Netral


In [21]:
def process_sentiment_labeling(df, client, batch_size=10, delay=90, checkpoint_file='labelling_progress.csv'):
    """
    Process sentiment labeling dengan auto-checkpoint untuk resume progress.
    
    Args:
        df: DataFrame dengan komentar
        client: Gemini client
        batch_size: Jumlah komentar per batch
        delay: Delay antar batch (detik)
        checkpoint_file: File untuk menyimpan progress
    """
    
    # Cek apakah ada checkpoint dari run sebelumnya
    start_index = 0
    all_labels = []
    
    if os.path.exists(checkpoint_file):
        print("🔄 Menemukan checkpoint dari run sebelumnya...")
        df_checkpoint = pd.read_csv(checkpoint_file)
        start_index = len(df_checkpoint)
        all_labels = df_checkpoint['sentiment'].tolist()
        print(f"✅ Resume dari index {start_index} ({start_index} komentar sudah selesai)")
        
        # Validasi: pastikan checkpoint cocok dengan dataframe
        if start_index >= len(df):
            print("✅ Semua data sudah selesai dilabeli!")
            return df_checkpoint
    else:
        print("🆕 Memulai labeling dari awal...")
    
    # Loop setiap batch komentar (mulai dari start_index)
    for i in range(start_index, len(df), batch_size):
        batch_num = i // batch_size + 1
        batch_comments = df['Teks_Komentar'].iloc[i:i+batch_size].astype(str).tolist()
        print(f"\n📦 Batch {batch_num} ({len(batch_comments)} komentar, index {i}-{i+len(batch_comments)-1})...")

        retry_count = 0
        max_retries = 3
        
        while retry_count < max_retries:
            try:
                labels = classify_sentiment_batch(batch_comments)
                
                # Cek apakah ada "Error" di hasil
                error_count = labels.count("Error")
                if error_count > 0:
                    print(f"⚠️ Ada {error_count} label Error, retry {retry_count + 1}/{max_retries}...")
                    retry_count += 1
                    if retry_count >= max_retries:
                        print(f"❌ Batch gagal setelah {max_retries} kali retry, menyimpan dengan label Error")
                        break
                    time.sleep(30)  # Wait before retry
                    continue
                    
                break  # Berhasil, keluar dari loop retry
                
            except Exception as e:
                retry_count += 1
                if "RESOURCE_EXHAUSTED" in str(e) or "429" in str(e):
                    wait_time = 180 * retry_count  # Exponential backoff
                    print(f"⚠️ Quota exhausted, retry {retry_count}/{max_retries} dalam {wait_time}s...")
                    if retry_count >= max_retries:
                        print("❌ Max retry tercapai, menyimpan progress dan berhenti.")
                        labels = ["Error"] * len(batch_comments)
                        break
                    time.sleep(wait_time)
                else:
                    print(f"❌ Error pada batch {batch_num}: {e}")
                    labels = ["Error"] * len(batch_comments)
                    break

        # Simpan hasil batch ini
        all_labels.extend(labels)
        
        # 💾 CHECKPOINT: Simpan progress setiap batch
        df_progress = pd.DataFrame({
            "Teks_Komentar": df["Teks_Komentar"].iloc[:len(all_labels)],
            "sentiment": all_labels
        })
        df_progress.to_csv(checkpoint_file, index=False, encoding="utf-8")
        print(f"💾 Checkpoint disimpan ({len(all_labels)}/{len(df)} selesai)")
        
        # Jeda antar batch
        if i + batch_size < len(df):
            print(f"⏳ Menunggu {delay} detik sebelum batch berikutnya...")
            time.sleep(delay)
    
    # Final result
    df_result = pd.DataFrame({
        "Teks_Komentar": df["Teks_Komentar"],
        "sentiment": all_labels
    })
    
    print("\n" + "="*50)
    print("✅ LABELING SELESAI!")
    print(f"Total data: {len(df_result)}")
    print(f"Positif: {all_labels.count('Positif')}")
    print(f"Negatif: {all_labels.count('Negatif')}")
    print(f"Netral: {all_labels.count('Netral')}")
    print(f"Error: {all_labels.count('Error')}")
    print("="*50)
    
    return df_result

In [22]:
df_result = process_sentiment_labeling(df, client, batch_size=50, delay=60)

🔄 Menemukan checkpoint dari run sebelumnya...
✅ Resume dari index 350 (350 komentar sudah selesai)

📦 Batch 8 (50 komentar, index 350-399)...
✅ Berhasil: gemini-2.5-flash (API key #2)
💾 Checkpoint disimpan (400/1072 selesai)
⏳ Menunggu 60 detik sebelum batch berikutnya...

📦 Batch 9 (50 komentar, index 400-449)...
✅ Berhasil: gemini-2.5-flash (API key #2)
💾 Checkpoint disimpan (450/1072 selesai)
⏳ Menunggu 60 detik sebelum batch berikutnya...

📦 Batch 10 (50 komentar, index 450-499)...
❌ gemini-2.5-flash: Error - 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}
⚠️ gemini-2.0-flash-lite-001 (API key #1): Quota exhausted
🔄 Switch ke API key #2
🔄 Retry gemini-2.0-flash-lite-001 dengan API key baru...
⚠️ gemini-2.0-flash-lite-001 (API key #2): Quota exhausted
❌ Semua API key habis untuk gemini-2.0-flash-lite-001, coba model lain...
⚠️ gemini-2.0-flash-exp (API key #1): Quota exhausted
🔄 Switch ke API key #2
🔄 

In [23]:
# Baca kedua file
df_cleaned = pd.read_csv("data_filtered.csv")

df_cleaned['sentiment'] = df_result['sentiment']

df_cleaned.to_csv("dataset_labeled.csv", index=False, encoding="utf-8")

print("Dataset final selesai disimpan ke dataset_labeled.csv")

Dataset final selesai disimpan ke dataset_labeled.csv
